## Máster en Big Data y Data Science

### Metodologías de gestión y diseño de proyectos de big data

#### AP2 - Preparación de los datos para clustering

---

En esta libreta se realizan las transforamciones sobre los datasets del escenario considerando su uso para los métodos de clusterización.

---

In [1]:
#Se importan las librerias a utilizar

import pandas as pd

----

##### Lectura de los datasets

Se parte de los mismos datasets con los que se realizó la integración anterior.

In [2]:
df_creditos = pd.read_csv("../../data/processed/datos_creditos_mc.csv", sep=";")
display(df_creditos.head(1))

df_tarjetas = pd.read_csv("../../data/processed/datos_tarjetas_mc.csv", sep=";")
display(df_tarjetas.head(1))

,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago
0,713061558.0,22,35000,3,123.0,ALQUILER,59000,PERSONAL,0.59,16.02,1,Y


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo
0,713061558.0,36.0,CASADO,ACTIVO,1088.0,M,4010.0,UNIVERSITARIO_COMPLETO,Blue,24.0,2.0


---
#### Aplicación de transformaciones

**Operaciones a realizar**

1. Selección de columnas
2. Filtrado de filas
3. Construcción de atributos
4. Integración de datasets
5. Formateo definitivo


----

Selección de datos

In [3]:
# Se establece qué columnas se eliminan

col_eliminar_creditos = ['antiguedad_empleado']
col_eliminar_tarjetas = ['limite_credito_tc', 'genero', 'personas_a_cargo', 'nivel_educativo', 'operaciones_ult_12m']

# Se ejecuta la operación

df_creditos.drop(col_eliminar_creditos, inplace=True, axis=1)
df_tarjetas.drop(col_eliminar_tarjetas, inplace=True, axis=1)

In [4]:
print("Vista del dataset de datos de créditos:")
display(df_creditos.head(1))

print("Vista del dataset de datos de tarjetas:")
display(df_tarjetas.head(1))

Vista del dataset de datos de créditos:


,id_cliente,edad,importe_solicitado,duracion_credito,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago
0,713061558.0,22,35000,3,ALQUILER,59000,PERSONAL,0.59,16.02,1,Y


Vista del dataset de datos de tarjetas:


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,nivel_tarjeta
0,713061558.0,36.0,CASADO,ACTIVO,1088.0,Blue


Limpieza de los datos (filtrado a nivel de filas)

In [5]:
#Se puede definir una función para aplicar los cálculos
def regla_pct_ingresos_credito(row):
    pct_ingreso = row.pct_ingreso
    ingresos = row.ingresos
    
    if pct_ingreso > 0.5 and ingresos <= 20000:
        # Es un error, no cumple la regla definida
        return 'err'
    else:
        return 'ok'


# Se aplica la función para todos los elementos del dataset
regla_pct_ingresos = df_creditos.apply(lambda row: regla_pct_ingresos_credito(row), axis=1).rename("regla_pct_ingresos")

# Se unen los resultados al dataset inicial
df_creditos = pd.concat([df_creditos, regla_pct_ingresos], axis=1)
df_creditos.head(5)  

,id_cliente,edad,importe_solicitado,duracion_credito,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,regla_pct_ingresos
0,713061558.0,22,35000,3,ALQUILER,59000,PERSONAL,0.59,16.02,1,Y,ok
1,768805383.0,21,1000,2,PROPIA,9600,EDUCACIÓN,0.10,11.14,0,N,ok
2,818770008.0,25,5500,3,HIPOTECA,9600,SALUD,0.57,12.87,1,N,err
3,713982108.0,23,35000,2,ALQUILER,65500,SALUD,0.53,15.23,1,N,ok
4,710821833.0,24,35000,4,ALQUILER,54400,SALUD,0.55,14.27,1,Y,ok


In [6]:
# Regla 2
#Se puede definir una función para aplicar los cálculos
def regla_pct_ingresos_credito_minimo_propiedad(row, min_valor):
    pct_ingreso = row.pct_ingreso
    tipo_propietario = row.situacion_vivienda
    dur_credito = row.duracion_credito
    
    if pct_ingreso > 0.6 and tipo_propietario != 'PROPIA' and min_valor == dur_credito:
        # Es un error, no cumple la regla definida
        return 'err'
    else:
        return 'ok'
    
# Como en la documentación no indica nada al respecto de cuanto es el mínimo permitido de duración de un crédito, tomaré el valor mínimo de la columna
min_valor = df_creditos['duracion_credito'].min()

# Se aplica la función para todos los elementos del dataset
regla_pct_ingresos = df_creditos.apply(lambda row: regla_pct_ingresos_credito_minimo_propiedad(row, min_valor), axis=1).rename("regla_pct_ingresos_credito_minimo_propiedad")

# Se unen los resultados al dataset inicial
df_creditos = pd.concat([df_creditos, regla_pct_ingresos], axis=1)
df_creditos.head(5)

,id_cliente,edad,importe_solicitado,duracion_credito,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,regla_pct_ingresos,regla_pct_ingresos_credito_minimo_propiedad
0,713061558.0,22,35000,3,ALQUILER,59000,PERSONAL,0.59,16.02,1,Y,ok,ok
1,768805383.0,21,1000,2,PROPIA,9600,EDUCACIÓN,0.10,11.14,0,N,ok,ok
2,818770008.0,25,5500,3,HIPOTECA,9600,SALUD,0.57,12.87,1,N,err,ok
3,713982108.0,23,35000,2,ALQUILER,65500,SALUD,0.53,15.23,1,N,ok,ok
4,710821833.0,24,35000,4,ALQUILER,54400,SALUD,0.55,14.27,1,Y,ok,ok


In [7]:
# Se filtran las filas con algún error detectado
print(f"Filas antes del filtro: {df_creditos.shape[0]}")

# Filtro descartando edades mayores a sesenta años
temp = df_creditos[df_creditos['edad'] < 90]

# Filtro descartando porcentajes mayores al 70 por cioento ya que sería irreal en un  caso real bancario
temp_a = temp[temp['pct_ingreso'] < 0.7]

# Otro filtro posible: por la regla de negocio agregada
temp_b = temp_a[temp_a['regla_pct_ingresos'] == 'ok']

#Otro filtro posible: por la segunda regla de negocio agregada
temp_c = temp_b[temp_b['regla_pct_ingresos_credito_minimo_propiedad'] == 'ok']

print(f"Filas después del filtro: {temp_c.shape[0]}")

Filas antes del filtro: 10127
Filas después del filtro: 10102


In [8]:
# Primero calculo el valor medio del atributo
val_med = temp_c['tasa_interes'].mean()

# Posteriorment asigno el valor medio del atribtuo a los nulos
temp_c['tasa_interes'].fillna(val_med, inplace=True)

# Verfico que he eliminado los valores nulos
nulls = temp_c['tasa_interes'].isnull().sum()

if nulls == 0:
    print("Se han substituido todos los valores nulos del atributo tasa_interes por la media correspondiente.")

Se han substituido todos los valores nulos del atributo tasa_interes por la media correspondiente.


C:\Users\victo\AppData\Local\Temp\ipykernel_2164\435584869.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  temp_c['tasa_interes'].fillna(val_med, inplace=True)
C:\Users\victo\AppData\Local\Temp\ipykernel_2164\435584869.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_c['tasa_interes'].fillna(val_med, inplace=True)


Integración de datos

In [9]:
df_integrado = pd.merge(temp_c, df_tarjetas, on='id_cliente', how='inner')
coincidencias = df_integrado.shape[0]

print(f"Filas del dataset integrado con los filtros realizados: {coincidencias}")

Filas del dataset integrado con los filtros realizados: 10102


In [10]:
print(f"Cantidad de columnas del dataset integrado: {df_integrado.shape[1]}")

Cantidad de columnas del dataset integrado: 18


#### Transformación de atributos

Observación general del dataset

In [11]:
df_integrado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10102 entries, 0 to 10101
Data columns (total 18 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   id_cliente                                   10102 non-null  float64
 1   edad                                         10102 non-null  int64  
 2   importe_solicitado                           10102 non-null  int64  
 3   duracion_credito                             10102 non-null  int64  
 4   situacion_vivienda                           10102 non-null  object 
 5   ingresos                                     10102 non-null  int64  
 6   objetivo_credito                             10102 non-null  object 
 7   pct_ingreso                                  10102 non-null  float64
 8   tasa_interes                                 10102 non-null  float64
 9   estado_credito                               10102 non-null  int64  
 10

##### Procesamiento de valores nulos

En este caso se van a filtrar (eliminar)

En mi caso especial no hace falta tratar los valores nulos. Debido a que la columna anitugedad_empleado la decidí eliminar en la actividad 1 y respecto a la tasa_interes, ya traté los valores nulos en el apartado anterior mediante la mediana del atributo, completando así los valores nulos.

In [12]:
# df_filtrado = df_integrado[(df_integrado['tasa_interes'].notnull()) & (df_integrado['antiguedad_empleado'].notnull())]
df_filtrado = df_integrado
# Se puede observar que no hay valores nulos
df_filtrado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10102 entries, 0 to 10101
Data columns (total 18 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   id_cliente                                   10102 non-null  float64
 1   edad                                         10102 non-null  int64  
 2   importe_solicitado                           10102 non-null  int64  
 3   duracion_credito                             10102 non-null  int64  
 4   situacion_vivienda                           10102 non-null  object 
 5   ingresos                                     10102 non-null  int64  
 6   objetivo_credito                             10102 non-null  object 
 7   pct_ingreso                                  10102 non-null  float64
 8   tasa_interes                                 10102 non-null  float64
 9   estado_credito                               10102 non-null  int64  
 10

##### Procesamiento de atributos nominales

Se realiza una binarización por aquellos métodos que no pueden operar con atributos no numéricos.

In [13]:
data = pd.get_dummies(df_filtrado)
data.head()

,id_cliente,edad,importe_solicitado,duracion_credito,ingresos,pct_ingreso,tasa_interes,estado_credito,antiguedad_cliente,gastos_ult_12m,...,estado_civil_CASADO,estado_civil_DESCONOCIDO,estado_civil_DIVORCIADO,estado_civil_SOLTERO,estado_cliente_ACTIVO,estado_cliente_PASIVO,nivel_tarjeta_Blue,nivel_tarjeta_Gold,nivel_tarjeta_Platinum,nivel_tarjeta_Silver
0,713061558.0,22,35000,3,59000,0.59,16.02,1,36.0,1088.0,...,True,False,False,False,True,False,True,False,False,False
1,768805383.0,21,1000,2,9600,0.10,11.14,0,39.0,1144.0,...,True,False,False,False,True,False,True,False,False,False
2,713982108.0,23,35000,2,65500,0.53,15.23,1,36.0,1887.0,...,True,False,False,False,True,False,True,False,False,False
3,710821833.0,24,35000,4,54400,0.55,14.27,1,54.0,1314.0,...,True,False,False,False,True,False,True,False,False,False
4,769911858.0,21,2500,2,9900,0.25,7.14,1,34.0,1171.0,...,False,True,False,False,True,False,True,False,False,False


In [14]:
# Se establece qué columnas se eliminan
# Elimino estas tres columnas debido a que ya han cumplido su objetivo por tanto ya se pueden eliminar

col_eliminar = ['id_cliente', 'regla_pct_ingresos_ok', 'regla_pct_ingresos_credito_minimo_propiedad_ok']

# Se ejecuta la operación

data.drop(col_eliminar, inplace=True, axis=1)

In [15]:
data.head(5)

,edad,importe_solicitado,duracion_credito,ingresos,pct_ingreso,tasa_interes,estado_credito,antiguedad_cliente,gastos_ult_12m,situacion_vivienda_ALQUILER,...,estado_civil_CASADO,estado_civil_DESCONOCIDO,estado_civil_DIVORCIADO,estado_civil_SOLTERO,estado_cliente_ACTIVO,estado_cliente_PASIVO,nivel_tarjeta_Blue,nivel_tarjeta_Gold,nivel_tarjeta_Platinum,nivel_tarjeta_Silver
0,22,35000,3,59000,0.59,16.02,1,36.0,1088.0,True,...,True,False,False,False,True,False,True,False,False,False
1,21,1000,2,9600,0.10,11.14,0,39.0,1144.0,False,...,True,False,False,False,True,False,True,False,False,False
2,23,35000,2,65500,0.53,15.23,1,36.0,1887.0,True,...,True,False,False,False,True,False,True,False,False,False
3,24,35000,4,54400,0.55,14.27,1,54.0,1314.0,True,...,True,False,False,False,True,False,True,False,False,False
4,21,2500,2,9900,0.25,7.14,1,34.0,1171.0,False,...,False,True,False,False,True,False,True,False,False,False


Se exportan los resultados para poder utilizarlos en las operaciones de clustering

In [16]:
data.to_csv("../../data/final/datos_clusterizacion.csv", sep=';', index=False)

In [17]:
print(f"Cantidad de filas del dataset integrado: {data.shape[0]}")
print(f"Cantidad de columnas del dataset integrado: {data.shape[1]}")

Cantidad de filas del dataset integrado: 10102
Cantidad de columnas del dataset integrado: 31
